In [1]:
# import random
# import preamble
# from robot import Robot
# from timer import Timer
# from virtual_dynamics import SimpleVirtualDynamics
# import numpy as np
# import time

# HOME_POSE = np.array([-0.3660424012449587, 0.6117162981347448, 0.9079274150612482, -0.005400995019868388, -0.0019643877311394183, -2.354619210713495,])

# # EMPIRACLLY FOUND BOUNDS LMAOOO
# # X_1 0.9424930299658596 0.6117557194238993 0.9078604285583273 -0.00548249567674523 -0.001965080545549638 -2.354599543904734
# # X_2 -0.9550680520089313 0.6117414180340556 0.9079711187002182 -0.0054476943775778695 -0.0019114211088729508 -2.354654218591263
# # Y_1 -0.3660752951621683 -0.9769698635830291 0.9078264973478847 -0.0053542629496214685 -0.001810702334644549 -2.3546477885984776
# # Y_2 -0.36601869093216893 1.0760272286463097 0.9079063215858276 -0.005390598302379138 -0.0020253279548708295 -2.3546219602941947
# # Z_1 -0.3660286022964023 0.6117127454372178 0.19288964682311052 -0.005344021094379553 -0.0020067450336018183 -2.3546172810944115
# # Z_2 -0.36607992182429566 0.611725276577193 1.3242422427347065 -0.005394167984549681 -0.00198225913263719 -2.35459296105138

# # ps = [
# #     np.array([0.9424930299658596, 0.6117557194238993, 0.9078604285583273,]),
# #     np.array([-0.9550680520089313, 0.6117414180340556, 0.9079711187002182,]),
# #     np.array([-0.3660752951621683, -0.9769698635830291, 0.9078264973478847,]),
# #     np.array([-0.36601869093216893, 1.0760272286463097, 0.9079063215858276,]),
# #     np.array([-0.3660286022964023, 0.6117127454372178, 0.19288964682311052,]),
# #     np.array([-0.36607992182429566, 0.611725276577193, 1.3242422427347065,]),
# # ]

# # for p in ps:
# #     print(np.linalg.norm(p - np.array([-0.3660424012449587, 0.6117162981347448, 0.9079274150612482,])))

# # print(0.4163148294610553 * 0.95)

# RADIUS = 0.35
# SPEED_BOUNDS = (0, 3)  # Replace n with the maximum speed
# ACCELERATION_BOUNDS = (0, 5)  # Replace m with the maximum acceleration

# def random_point_in_sphere(center, radius):
#     # Generate a random direction (unit vector)
#     direction = np.random.normal(0, 1, size=center.shape)
#     direction /= np.linalg.norm(direction)  # Normalize to unit vector

#     # Generate a random radius within [0, radius]
#     random_radius = np.cbrt(np.random.uniform(0, 1)) * radius

#     # Compute the random point
#     random_point = center + random_radius * direction
#     return random_point

# with Robot(
#     '169.254.9.43',
#     translational_force_deadband=6.0,
#     rotational_force_deadband=0.5,
#     init_pose=HOME_POSE
# ) as r:
    
#     while True:
        # random_speed = random.uniform(*SPEED_BOUNDS)
        # random_acceleration = random.uniform(*ACCELERATION_BOUNDS)
        # r.set_pose(random_point_in_sphere(np.array([-0.3660424012449587, 0.6117162981347448, 0.9079274150612482,]), RADIUS), Robot.TRANSLATION, speed=random_speed, acceleration=random_acceleration)
#         time.sleep(0.1)
#         r.set_pose(np.array([-0.3660424012449587, 0.6117162981347448, 0.9079274150612482,]), Robot.TRANSLATION)
#         time.sleep(0.1)

#     # AXES = Robot.TRANSLATION

#     # timer = Timer()

#     # vd = SimpleVirtualDynamics(
#     #     M=10,
#     #     B=25,
#     #     K=0,
#     # )

#     # while True:
#     #     period_start = r.control.initPeriod()

#     #     time = timer.t()

#     #     dt = timer.dt()

#     #     position = r.get_pose(Robot.TRANSLATION_ROTATION)

#     #     velocity = r.get_velocity(AXES)

#     #     force = r.get_force(AXES)

#     #     vd.apply_force(force, dt)

#     #     r.set_velocity(vd.get_velocity(), AXES, acceleration=10)

#     #     r.control.waitPeriod(period_start)

In [2]:
# import preamble
# import numpy as np
# from robot import Robot
# from timer import Timer
# from phrase_trial_data import PhraseTrialData

# NUMBER_OF_USERS = 12
# NUMBER_OF_TRIALS_PER_USER = 12

# USER_ID = 1

# current_trial_number = 1

# NUM_OF_STAGES = 5
# stage = 0
# def next_stage():
#     global stage
#     stage = (stage + 1) % NUM_OF_STAGES

# HOME_POSE = np.array([-0.3660424012449587, 0.6117162981347448, 0.9079274150612482, -0.005400995019868388, -0.0019643877311394183, -2.354619210713495,])

# with Robot(
#     '169.254.9.43',
#     translational_force_deadband=6.0,
#     rotational_force_deadband=0.5,
#     init_pose=HOME_POSE
# ) as r:
#     AXES = Robot.TRANSLATION

#     timer = Timer()

#     while True:
#         period_start = r.control.initPeriod()

#         if stage == 0: # wait to begin trial
#             cmd = input()
#             if cmd == 'exit':
#                 break
#             next_stage()
#         elif stage == 1: # trial initialization
#             phrase_trial_data = PhraseTrialData(USER_ID, current_trial_number, '', '', '', '')
#             timer.reset()
#             next_stage()
#         elif stage == 2: # trial recording
#             time = timer.t()

#             dt = timer.dt()

#             position = r.get_pose(AXES)

#             velocity = r.get_velocity(AXES)

#             force = r.get_force(AXES)

#             phrase_trial_data.append(
#                 time,
#                 dt,
#                 position,
#                 velocity,
#                 force,
#             )

#             task_frame = [0, 0, 0, 0, 0, 0]
#             selection_vector = [0, 0, 1, 0, 0, 0]
#             wrench = [0, 0, 100, 0, 0, 0]
#             force_type = 2
#             limits = [2, 2, 1.5, 1, 1, 1]
#             r.control.forceMode(task_frame, selection_vector, wrench, force_type, limits)

#             if time > 2:
#                 task_frame = [0, 0, 0, 0, 0, 0]
#                 selection_vector = [0, 0, 1, 0, 0, 0]
#                 wrench = [0, 0, 0, 0, 0, 0]
#                 force_type = 2
#                 limits = [2, 2, 1.5, 1, 1, 1]
#                 r.control.forceMode(task_frame, selection_vector, wrench, force_type, limits)
#                 r.control.forceModeStop()
#                 next_stage()
#         elif stage == 3: # get user feedback, restart, or exit
#             cmd = input()
#             if cmd == 'exit':
#                 break
#             elif cmd == 'restart':
#                 pass
#             else:
#                 phrase_text, adverb, first_cartesian_direction, second_cartesian_direction = cmd.split(',')
#                 phrase_trial_data.phrase = phrase_text
#                 phrase_trial_data.adverb = adverb
#                 phrase_trial_data.first_cartesian_direction = first_cartesian_direction
#                 phrase_trial_data.second_cartesian_direction = second_cartesian_direction
#                 phrase_trial_data.save(current_trial_number, 'force2lang_trial_data')
#                 current_trial_number += 1
#             next_stage()
#         elif stage == 4: # return to home pose
#             position = r.get_pose(AXES)
#             position_delta = Robot.get_axes(r.INIT_POSE, AXES) - position
#             position_delta_magnitude = np.linalg.norm(position_delta)
#             position_delta_normalized = position_delta / position_delta_magnitude if position_delta_magnitude != 0 else 0
#             return_velocity = 0.2
#             r.set_velocity(return_velocity*position_delta_normalized, AXES, acceleration=2)
#             if position_delta_magnitude < 0.001:
#                 r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
#                 next_stage()

#     r.control.waitPeriod(period_start)

# print(current_trial_number)

In [3]:
import preamble
import numpy as np
from robot import Robot
from timer import Timer
from phrase_trial_data import PhraseTrialData
import random

NUMBER_OF_USERS = 12
NUMBER_OF_TRIALS_PER_USER = 12

USER_ID = 1

current_trial_number = 1

NUM_OF_STAGES = 5
stage = 0
def next_stage():
    global stage
    stage = (stage + 1) % NUM_OF_STAGES

RADIUS = 0.35
SPEED_BOUNDS = (0.2, 0.8)  # Replace n with the maximum speed
ACCELERATION_BOUNDS = (0.2, 0.4)  # Replace m with the maximum acceleration

def random_point_in_sphere(center, radius):
    # Generate a random direction (unit vector)
    direction = np.random.normal(0, 1, size=center.shape)
    direction /= np.linalg.norm(direction)  # Normalize to unit vector

    # Generate a random radius within [0, radius]
    random_radius = np.cbrt(np.random.uniform(0, 1)) * radius

    # Compute the random point
    random_point = center + random_radius * direction
    return random_point

TIME_STEP = 0.01

def generate_velocity_curve(start_position, target_position, max_velocity, max_acceleration, time_step=0.01):
    # Calculate distance and direction
    position_delta = target_position - start_position
    total_distance = np.linalg.norm(position_delta)
    direction = position_delta / total_distance

    # Divide total time into thirds
    total_time = 3 * np.sqrt(total_distance / max_acceleration)  # Adjust total time based on distance and acceleration
    t_ramp = total_time / 3
    t_const = total_time / 3

    # Adjust max velocity to fit the divided time profile
    max_velocity = max_acceleration * t_ramp

    # Generate velocity curve
    time = 0
    velocities = []
    while time < total_time:
        if time <= t_ramp:  # Ramp-up phase
            velocity_magnitude = max_acceleration * time
        elif time <= t_ramp + t_const:  # Constant velocity phase
            velocity_magnitude = max_velocity
        else:  # Ramp-down phase
            velocity_magnitude = max_velocity - max_acceleration * (time - t_ramp - t_const)

        velocity = direction * velocity_magnitude
        velocities.append(velocity)
        time += time_step

    return np.array(velocities), total_time

HOME_POSE = np.array([-0.3660424012449587, 0.6117162981347448, 0.9079274150612482, -0.005400995019868388, -0.0019643877311394183, -2.354619210713495,])

with Robot(
    '169.254.9.43',
    translational_force_deadband=6.0,
    rotational_force_deadband=0.5,
    init_pose=HOME_POSE
) as r:
    AXES = Robot.TRANSLATION

    timer = Timer()

    while True:
        period_start = r.control.initPeriod()

        if stage == 0: # wait to begin trial
            cmd = input()
            if cmd == 'exit':
                break
            next_stage()
        elif stage == 1: # trial initialization
            phrase_trial_data = PhraseTrialData(USER_ID, current_trial_number, '', '', '', '')
            random_speed = random.uniform(*SPEED_BOUNDS)
            random_acceleration = random.uniform(*ACCELERATION_BOUNDS)
            random_point = random_point_in_sphere(Robot.get_axes(r.INIT_POSE, AXES), RADIUS)
            
            # Generate the velocity curve
            velocity_curve, total_time = generate_velocity_curve(
                start_position=Robot.get_axes(r.INIT_POSE, AXES),
                target_position=random_point,
                max_velocity=random_speed,
                max_acceleration=random_acceleration,
                time_step=TIME_STEP
            )
            print(total_time)
            curve_index = 1  # Initialize curve index

            timer.reset()
            last_update_time = timer.t()
            r.set_velocity(velocity_curve[0], AXES, acceleration=random_acceleration)
            next_stage()
        elif stage == 2: # trial recording
            time = timer.t()

            dt = timer.dt()

            position = r.get_pose(AXES)

            velocity = r.get_velocity(AXES)

            force = r.get_force(AXES)

            phrase_trial_data.append(
                time,
                dt,
                position,
                velocity,
                force,
            )

            if curve_index < len(velocity_curve):
                if time - last_update_time >= TIME_STEP:
                    r.set_velocity(velocity_curve[curve_index], AXES, acceleration=random_acceleration)
                    curve_index += 1
            else:
                r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
                next_stage()

            # position_delta = random_point - position
            # position_delta_magnitude = np.linalg.norm(position_delta)
            # position_delta_normalized = position_delta / position_delta_magnitude if position_delta_magnitude != 0 else 0
            # return_velocity = random_speed
            # r.set_velocity(return_velocity*position_delta_normalized, AXES, acceleration=random_acceleration)
            # if position_delta_magnitude < 0.001:
            #     r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
            #     next_stage()

            # task_frame = [0, 0, 0, 0, 0, 0]
            # selection_vector = [0, 0, 1, 0, 0, 0]
            # wrench = [0, 0, 100, 0, 0, 0]
            # force_type = 2
            # limits = [2, 2, 1.5, 1, 1, 1]
            # r.control.forceMode(task_frame, selection_vector, wrench, force_type, limits)

            # if time > 2:
                # task_frame = [0, 0, 0, 0, 0, 0]
                # selection_vector = [0, 0, 1, 0, 0, 0]
                # wrench = [0, 0, 0, 0, 0, 0]
                # force_type = 2
                # limits = [2, 2, 1.5, 1, 1, 1]
                # r.control.forceMode(task_frame, selection_vector, wrench, force_type, limits)
                # r.control.forceModeStop()
                # next_stage()
        elif stage == 3: # get user feedback, restart, or exit
            cmd = input()
            if cmd == 'exit':
                break
            elif cmd == 'restart':
                pass
            else:
                phrase_text, adverb, first_cartesian_direction, second_cartesian_direction = cmd.split(',')
                phrase_trial_data.phrase = phrase_text
                phrase_trial_data.adverb = adverb
                phrase_trial_data.first_cartesian_direction = first_cartesian_direction
                phrase_trial_data.second_cartesian_direction = second_cartesian_direction
                phrase_trial_data.save(current_trial_number, 'force2lang_trial_data')
                current_trial_number += 1
            next_stage()
        elif stage == 4: # return to home pose
            position = r.get_pose(AXES)
            position_delta = Robot.get_axes(r.INIT_POSE, AXES) - position
            position_delta_magnitude = np.linalg.norm(position_delta)
            position_delta_normalized = position_delta / position_delta_magnitude if position_delta_magnitude != 0 else 0
            return_velocity = 0.2
            r.set_velocity(return_velocity*position_delta_normalized, AXES, acceleration=2)
            if position_delta_magnitude < 0.001:
                r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
                next_stage()

    r.control.waitPeriod(period_start)

print(current_trial_number)

2.4603876541921874
3.2849919019724867
2.519680541266375
1


In [4]:
ANGLE = 5.0 * np.pi / 4.0
TRANSFORMATION = np.array([
    [np.cos(ANGLE), -np.sin(ANGLE), 0.0],
    [np.sin(ANGLE),  np.sin(ANGLE), 0.0],
    [0.0, 0.0, 1.0]
])

p = PhraseTrialData.load('./force2lang_trial_data/1__5__.pkl', TRANSFORMATION)

import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(8, 5))
plt.plot(p.time, p.velocity, label="Velocity over Time")

# Add labels and title
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.title("Time vs. Velocity")
plt.legend()

# Display the grid and the plot
plt.grid(True)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './force2lang_trial_data/1__5__.pkl'